In [1]:
# Import essential packages
import wntr
import pickle
import networkx as nx
import numpy as np
import copy

In [2]:
BWdn = wntr.network.WaterNetworkModel(r'CoH_Backbone_Final_O.inp')

C:\Users\Zhouxiaomu\anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "CoH_Backbone_Final_O.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [3]:
Wdn = wntr.network.WaterNetworkModel(r'CoH_460MGD_2018.inp')

C:\Users\Zhouxiaomu\anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "CoH_460MGD_2018.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [4]:
# Simulate for a stable period
Wdn.options.time.duration = 24 * 3600
Sim = wntr.sim.EpanetSimulator(Wdn)
Results = Sim.run_sim()

In [5]:
# Get the important indicators
Flow = Results.link['flowrate']

In [6]:
# Build hourly hydraulic-informed directed graph 
def hourly_dgraph(wdn, flow, t): 
    # Initiated the directed graph
    dgraph = nx.DiGraph()
    # Use link (pipe, valve) diameter as the main selection criteria
    for name, link in wdn.links():
        # Determine the direction of edge through the flow rate, positive: as stored, negative: change direction
        linkflow = flow.loc[3600*t, name]
        # Only establish graph with nonzero flows
        if linkflow > 1e-8:
            start_node = link.start_node_name
            end_node = link.end_node_name
        elif linkflow < (-1e-8):
            start_node = link.end_node_name
            end_node = link.start_node_name
        edgelist = list(dgraph.edges)
        # Build corresponding original network
        # If there is already a link from start_node to end_node in the graph, then this is the parallel links (pumps)
        if (start_node, end_node) in edgelist:
            # Keep the existing flow 
            linkcapacity = dgraph[start_node][end_node]['capacity']
            dgraph.add_edge(start_node, end_node, linkid = name, capacity= linkcapacity + abs(linkflow))
        else:
            dgraph.add_node(start_node, pos = wdn.get_node(start_node).coordinates)
            dgraph.add_node(end_node, pos = wdn.get_node(end_node).coordinates)
            dgraph.add_edge(start_node, end_node, linkid = name, capacity= abs(linkflow))    
    return dgraph

In [7]:
T = 7
Dgraph = hourly_dgraph(Wdn, Flow, T)

In [8]:
All_nodes = Wdn.node_name_list
Backbone_nodes = BWdn.node_name_list
Removed_nodes = [Node for Node in All_nodes if Node not in Backbone_nodes]

### Get the locally supplied nodes and calculate fraction of their source of supply

In [46]:
All_sources = Wdn.reservoir_name_list
Global_sources = BWdn.reservoir_name_list
Local_sources = [r for r in All_sources if r not in Global_sources]

In [12]:
def get_supply(sources, wdn, flow, t):
    source_link = {}
    supply = 0
    for name, link in wdn.links():
        s = link.start_node_name
        if s in sources:
            supply += flow.loc[t*3600, name]    
    return supply 

In [47]:
Local_supply = get_supply(Local_sources, Wdn, Flow, T)
Local_supply

2.8397102607542

In [14]:
All_supply = get_supply(All_sources, Wdn, Flow, T)
All_supply

21.83695864213047

In [15]:
def get_tank_supply(tank_list, wdn, flow, t):
    tank_supply = 0
    for name, link in wdn.links():
        s = link.start_node_name
        e = link.end_node_name
        if s in tank_list:
            # Calculate the flow out
            tank_supply += flow.loc[t*3600, name]
        if e in tank_list:
            # Calculate the flow in
            tank_supply -= flow.loc[t*3600, name]
    return tank_supply 

In [16]:
BTank_list = BWdn.tank_name_list
Tank_list = Wdn.tank_name_list
Tank_supply = get_tank_supply(Tank_list, Wdn, Flow, T)
Tank_supply

3.902531898043444

In [44]:
BTank_supply = get_tank_supply(BTank_list, Wdn, Flow, T)
BTank_supply

2.5811331590975435

In [45]:
Removed_tanks = [t for t in Tank_list if t not in BTank_list]
RTank_supply = get_tank_supply(Removed_tanks, Wdn, Flow, T)
RTank_supply

1.321398738945901

In [48]:
print(25.735-1.321-2.840)

21.573999999999998


In [19]:
Global_supply = get_supply(Global_sources, Wdn, Flow, T)
Global_supply

18.997248381376266

In [20]:
Demand = Results.node['demand']

In [21]:
Junction_list = Wdn.junction_name_list

In [22]:
Junction_demand = {}
for j in Junction_list:
    Junction_demand[j] = Demand.loc[3600*T, j]

In [23]:
print(sum(Junction_demand.values()))

25.735289887632234


In [24]:
Removed_tanks = [t for t in Tank_list if t not in BTank_list]

In [25]:
Cycles = list(nx.simple_cycles(Dgraph))

In [41]:
Cyclenodes = list(set([node for c in Cycles for node in c]))

In [40]:
def aggregate(removed_nodes, backbone_nodes, cycle_downnodes, junction_demand, dgraph):
    # If there is still demand to aggegrate
    node_demand = {}
    # In case some nodes are not junctions 
    node_list = list(dgraph.nodes)
    junction_list = list(junction_demand.keys())
    for node in node_list:
        if node in junction_list:
            node_demand[node] = junction_demand[node]
        else:
            node_demand[node] = 0    
    dnetwork = copy.deepcopy(dgraph)
    # Only aggregate junctions with demands
    downstreams = [rn for rn in removed_nodes if (rn in junction_list and rn in node_list)]
    # Initialize the downstreamspre
    downstreamspre = downstreams
    downstreamspost = []
    # while we can perform aggregating and removing
    
    # First deal with all the simple nodes
    while (len(downstreamspre)-len(downstreamspost)) > 0:
        downstreamspre = copy.deepcopy(downstreams)
        downstreams_updated = copy.deepcopy(downstreams) 
        for node in downstreams:
            if node not in cycle_downnodes:
                # Only aggegrate nodes that have no sucessors
                nodesucc = list(dnetwork.successors(node))
                if len(nodesucc) == 0:
                    # Aggeragated its deamnd to its immediate upstream nodes
                    nodeprede = list(dnetwork.predecessors(node))
                    q = []
                    for k in range(len(nodeprede)):
                        # Get the flow from k to j, which is the capacity established before
                        q.append(dnetwork[nodeprede[k]][node]['capacity'])
                    q = np.array(q)
                    sumq = sum(q)
                    for k in range(len(nodeprede)):
                        node_demand[nodeprede[k]] += (q[k]/sumq)*node_demand[node]
                    downstreams_updated.remove(node)
                    dnetwork.remove_node(node)
        downstreams = copy.deepcopy(downstreams_updated)
        downstreamspost = copy.deepcopy(downstreams_updated)
    # Then deal with nodes in the cycles
    downstreamspost = []
    while (len(downstreamspre)-len(downstreamspost)) > 0:
        downstreamspre = copy.deepcopy(downstreams)
        downstreams_updated = copy.deepcopy(downstreams) 
        for node in downstreams:
            if node in cycle_downnodes:
                nodesucc = list(dnetwork.successors(node))
                upstreams = list(dnetwork.predecessors(node))
                # Start from the simple nodes to break the cycle
                if (len(nodesucc) == 1 and len(upstreams) == 1):
                    # Aggeragated its deamnd to its immediate upstream and downstream nodes
                    node_demand[nodesucc[0]] += node_demand[node]/2
                    node_demand[upstreams[0]] += node_demand[node]/2
                    downstreams_updated.remove(node)
                    dnetwork.remove_node(node)
                    cycle_downnodes.remove(node)
                if (len(nodesucc) == 1 and len(upstreams) == 0):
                    node_demand[nodesucc[0]] += node_demand[node]
                    downstreams_updated.remove(node)
                    dnetwork.remove_node(node)
                    cycle_downnodes.remove(node)
                if len(nodesucc) == 0:
                    nodeprede = list(dnetwork.predecessors(node))
                    q = []
                    for k in range(len(nodeprede)):
                        # Get the flow from k to j, which is the capacity established before
                        q.append(dnetwork[nodeprede[k]][node]['capacity'])
                    q = np.array(q)
                    sumq = sum(q)
                    for k in range(len(nodeprede)):
                        node_demand[nodeprede[k]] += (q[k]/sumq)*node_demand[node]
                    downstreams_updated.remove(node)
                    dnetwork.remove_node(node)
                    cycle_downnodes.remove(node)
        downstreams = copy.deepcopy(downstreams_updated)
        downstreamspost = copy.deepcopy(downstreams_updated)
    # Deal with the leftover nodes
    ldownstreamspre = copy.deepcopy(downstreams)
    ldownstreamspost = []
    while (len(ldownstreamspre)-len(ldownstreamspost)) > 0:
        ldownstreamspre = copy.deepcopy(downstreams)
        ldownstreams_updated = copy.deepcopy(downstreams)
        for node in downstreams:
            nodesucc = list(dnetwork.successors(node))
            # For the node that is both upstream and downstream of backbone nodes
            if (set(nodesucc).issubset(set(backbone_nodes)) or len(nodesucc) == 0):
                nodeprede = list(dnetwork.predecessors(node))
                q = []
                for k in range(len(nodeprede)):
                    # Get the flow from k to j, which is the capacity established before
                    q.append(dnetwork[nodeprede[k]][node]['capacity'])
                q = np.array(q)
                sumq = sum(q)
                for k in range(len(nodeprede)):
                    node_demand[nodeprede[k]] += (q[k]/sumq)*node_demand[node]
                ldownstreams_updated.remove(node)
                dnetwork.remove_node(node)
        downstreams = copy.deepcopy(ldownstreams_updated)
        ldownstreamspost = copy.deepcopy(ldownstreams_updated)
    backbone_demand = {}
    print(downstreams)
    for bn in backbone_nodes:
        backbone_demand[bn] = node_demand[bn]
    return backbone_demand, dnetwork, node_demand              

In [42]:
Backbone_demand, Dnetwork, Node_demand = aggregate(Removed_nodes, Backbone_nodes, Cyclenodes, Junction_demand, Dgraph)

['129', '130', '233', '242', '243', '365', '366', '367', '368', '375', '376', '379', '380', '381', '382', '383', '384', '385', '386', '390', '391', '424', '425', '434', '435', '436', '437', '438', '439', '440', '441', '443', '446', '447', '450', '451', '452', '453', '456', '457', '466', '467', '470', '471', '472', '473', '476', '477', '482', '483', '8092', '8152', '8138', '8126', '8120', '8158', '8172', '8164', '8166', '8144', '8128', '8142', '8132', '8086', '8090']


In [43]:
print(sum(Backbone_demand.values()))

20.588478115112597


In [30]:
# Withdraw the backbone node that is not junction but assigned with demand
for key, value in Backbone_demand.items():
    if (key not in Junction_list) and value > 0:
        print(key)

In [31]:
# Remove the non-junctions
Non_junctions = Global_sources + BTank_list
for key in Non_junctions:
    if key in Backbone_demand.keys():
        del Backbone_demand[key]

### Write to inp file

In [37]:
def write_junctions(filename, bwdn, demand_hourly):
    with open(filename, "w") as file:
        for key, value in demand_hourly.items():
            junctionobject = bwdn.get_node(key)
            file.write(key)
            # To keep the string left aligned
            spacelength = (20 - len(key))
            for k in range(spacelength):
                file.write(' ')
            # Write the elevation value We need to transfer the unit
            file.write(str(np.around((junctionobject.elevation * 3.28084), decimals=2)))
            spacelength = (20 - len(str(np.around((junctionobject.elevation * 3.28084), decimals=2))))
            for k in range(spacelength):
                file.write(' ')
            # Write the basline value, transfer to gpm
            file.write(str(np.around((demand_hourly[key] * 15850.3*1.61), decimals=2)))
            spacelength = (20 - len(str(np.around((demand_hourly[key] * 15850.3*1.61), decimals=2))))
            for k in range(spacelength):
                file.write(' ')
            # Write the patterns
            file.write( 'Pat0' + ';\n')

In [38]:
write_junctions('Aggregate_O_demand_7am.inp', BWdn, Backbone_demand)

In [25]:
def backbone_down_remove(backbone_nodes,removed_nodes, dgraph):
    backbone_remove = {}
    for i in range(len(backbone_nodes)):
        upstreams = []
        alldownstreams = []
        upstreams.append(backbone_nodes[i])
        for node in upstreams:
            try:
                node_successors = list(dgraph.successors(node))
                for d in node_successors:
                    # If there is still downstream neigbors in the removed_nodes
                    if d in removed_nodes:
                        # To make sure that the nodes in cycles will not to be visisted twice
                        if d not in upstreams:
                            upstreams.append(d)
                            alldownstreams.append(d)
            except:
                pass
        backbone_remove[backbone_nodes[i]] = alldownstreams
    return backbone_remove         

In [26]:
Backbone_remove = backbone_down_remove(Backbone_nodes, Removed_nodes, Dgraph)

In [27]:
def map_back(backbone_remove, removed_nodes):
    remove_backbone = {}
    for node in removed_nodes:
        remove_backbone[node] = []
    for key, value in backbone_remove.items():
        if len(value) > 0:
            for i in value:
                remove_backbone[i].append(key)
    return remove_backbone       

In [28]:
Remove_backbone = map_back(Backbone_remove, Removed_nodes)

In [59]:
# Obtain the graph that only include the backbone nodes and the removed_nodes and removed_links
def partial_graph(backbone_network, dgraph):
    whoeldgraph = copy.deepcopy(dgraph)
    # Get the edges in backbone_network
    backbone_edges = list(backbone_network.edges())
    whole_edges = list(dgraph.edges())
    for edge in backbone_edges:
        (sn, en) = edge
        linkid = backbone_network[sn][en]['linkid']
        for edgew in whole_edges:
            (snw, enw) = edgew
            if linkid == dgraph[snw][enw]['linkid']:
                whoeldgraph.remove_edge(snw, enw)
    return whoeldgraph 

In [57]:
# Load the backbone network
FBackbone_network = open(r'CoH_Backbone_network.pickle', 'rb')
Backbone_network = pickle.load(FBackbone_network)
FBackbone_network.close()

In [60]:
Pgraph = partial_graph(Backbone_network, Dgraph)

In [62]:
def backtrackalgorithm(backbone_remove, dgraph):
    r = {}
    maximumdistance = {}
    # For each backbone node find its downstream removed nodes 
    for i, jlist in backbone_remove.items():
        r[i] = {}
        maximumdistance[i] = {}
        # Intialize the and rij
        r[i][i] = 1
        allnodes = jlist + [i] 
        # If the node has downstream removed nodes
        if len(jlist) > 0:
            for j in jlist:
                # j is accessible from i 
                paths = list(nx.all_simple_paths(dgraph, source=i, target=j))
                maximumd = 0
                for p in paths:
                    d = len(p)
                    if d > maximumd:
                        maximumd = d
                maximumdistance[i][j] = maximumd
        # Rank the nodes according to their maximum distance to the source
        nodetoevaluate = sorted(maximumdistance[i], key=maximumdistance[i].get)
        for j in nodetoevaluate:
            r[i][j] = 0
            # Get its upstream nodes that belong to the nodes to consider forward tracking
            klist = list(dgraph.predecessors(j))
            # As for the nodes that are not accessible from i, initialize its r[i][k] = 0
            for k in klist:
                if k not in allnodes:
                    r[i][k] = 0
            qkj = []
            for ik in range(len(klist)):
                # Get the flow from k to j, which is the capacity established before
                qkj.append(dgraph[klist[ik]][j]['capacity'])
            qkj = np.array(qkj)
            rkj = qkj/sum(qkj)
            for ik in range(len(klist)):
                r[i][j] += r[i][klist[ik]]*rkj[ik]
    return r

In [ ]:
R = backtrackalgorithm(Backbone_remove, Pgraph)

In [36]:
def allocate_demand(backbone_nodes, remove_backbone, junction_demand):
    node_allocation = {}
    junction_list = list(junction_demand.keys())
    for node in backbone_nodes:
        if node in junction_list:
            # Initialize the allocation as its original value
            node_allocation[node] = junction_demand[node]
        else:
            node_allocation[node] = 0
    for remove, value in remove_backbone.items():
        if len(value) > 0:
            if remove in junction_list:
                remove_demand = junction_demand[remove]
            else:
                remove_demand = 0
            for i in value:
                # Allocate the demand averagily to its upstream nodes
                node_allocation[i] += remove_demand/len(value)
    return node_allocation

In [75]:
print(sum(Junction_demand.values()))

21.258907076022005


In [83]:
Node_allocation = allocate_demand(Backbone_nodes, Remove_backbone, Junction_demand)

In [84]:
print(sum(Node_allocation.values()))

20.428286327073426


In [85]:
# Remove the reservoir nodes
for key in Global_sources:
    del Node_allocation[key]

In [88]:
# Remove the tank nodes
for key in BTank_list:
    del Node_allocation[key]

In [78]:
(20.428286327073426/25.735289929132477)

0.7937849693291585

In [ ]:
def backtrackalgorithm(backbone_remove, dgraph):
    r = {}
    # For each backbone node find its downstream removed nodes 
    for i, jlist in backbone_remove.items():
        r[i] = {}
        print(i, jlist)
        # Intialize the and rij
        r[i][i] = 1
        # If the node has downstream removed nodes
        if len(jlist) > 0:
            # All the nodes to consider for our forward tracking 
            allnodes = [i] + jlist
            jlength = len(jlist)
            # print(jlist)
            # Range the j list according to the downstream level
            for j in jlist:
                j_predecessors = list(dgraph.predecessors(j))
                for jp in j_predecessors:
                    # If its updtream node is also in the jlist, check its index
                    if jp in jlist:
                        jpindex, jindex = jlist.index(jp), jlist.index(j)
                        # If its upstream is after it 
                        if jpindex > jindex:
                            # Inset this element behind its upstream node
                            jlist.insert((jpindex+1), j)
            # Need to withdraw the last jlength of elements in jlist
            jlist_updated = jlist[(-jlength):]
            for j in jlist_updated:
                r[i][j] = 0
                # Get its upstream nodes that belong to the nodes to consider forward tracking 
                allpredecessors = list(dgraph.predecessors(j))
                klist = [node for node in allpredecessors if node in allnodes]
                qj = 0
                qkj = []
                for ik in range(len(klist)):
                    # Get the flow from k to j, which is the capacity established before
                    qkj.append(dgraph[klist[ik]][j]['capacity'])
                qkj = np.array(qkj)
                rkj = qkj/sum(qkj)
                for ik in range(len(klist)):
                    r[i][j] += r[i][klist[ik]]*rkj[ik]
    return r

In [ ]:
# Get all the nodes that have combined supply
def get_combined_junctions(local_sources, global_sources, dgraph):
    local_junctions = []
    for ls in local_sources:
        try:
            ls_des = nx.descendants(dgraph, ls)
            local_junctions += ls_des
        except:
            pass
    global_junctions = []
    for gs in global_sources:
        try:
            gs_des = nx.descendants(dgraph, gs)
            global_junctions += gs_des
        except:
            pass
    local_junctions = set(local_junctions)
    global_junctions = set(global_junctions)
    combined_junctions = list(local_junctions.intersection(global_junctions))
    return combined_junctions 

In [ ]:
Tseries = np.linspace(0, 23, 24)
S1 = 0
S2 = 0
D = 0
for t in Tseries:
    S1 += get_local_supply(All_sources, Wdn, Flow, t)
    S2 += get_tank_supply(Tank_list, Wdn, Flow, t)
    Junction_demand = {}
    for j in Junction_list:
        Junction_demand[j] = Demand.loc[3600*t, j]
    D += (sum(Junction_demand.values())) 

In [ ]:
Expected = wntr.metrics.hydraulic.expected_demand(Wdn)

In [ ]:
WSA = wntr.metrics.hydraulic.water_service_availability(Expected, Demand)

In [3]:
print((11.121+5.372+2.140))

18.633000000000003


In [4]:
print(11.402+4.804+2.158)

18.364


In [5]:
print((18.364-18.633)/18.633)

-0.01443675199914122
